In [38]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv('.\材料報價總覽 - 001.csv')
target = pd.read_csv('.\材料報價總覽 - Target.csv')

In [39]:
print('original data :',data.shape)

data = pd.concat([data, target], 0)
data.reset_index(inplace=True, drop=True)
print('concat data :',data.shape)
data.tail()

original data : (170, 61)
concat data : (171, 61)


,報價日期,報價廠商,專案名,數量,外盒,外盒材質,外盒印刷,外盒加工,外盒單價,外盒新刀,...,配件單價,配件.1,配件材質.1,配件加工.1,配件數量.1,配件單價.1,包裝單價,外箱單價,專案單價,專案總價
166,2019-3-25,墨帥,漢堡,500.0,70x70x60,圓筒盒,4c,NaN,50.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,2.3,152.3,76150.0
167,2019-3-25,墨帥,漢堡,1000.0,70x70x60,圓筒盒,4c,NaN,44.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,2.3,121.3,121300.0
168,2019-1-14,墨帥,漢堡,500.0,100x100x80,圓筒盒,4c,NaN,52.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13.0,2.3,151.3,75650.0
169,2019-1-14,墨帥,漢堡,1000.0,100x100x80,圓筒盒,4c,NaN,43.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,2.3,115.8,115800.0
170,2019-12-19,墨帥,兵姬中文版,500.0,224x224x38,天地盒,4c,水消+局部光,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
index = data.iloc[:, :4]
index.head(3)

,報價日期,報價廠商,專案名,數量
0,2019-12-17,墨帥,MOTT,500.0
1,2019-12-05,墨帥,原子怪獸,1000.0
2,2019-12-05,墨帥,原子怪獸,1500.0


In [41]:
#擷取兩份卡片資料表
card = data.iloc[:, 13:21]
card2 = data.iloc[:, 21:29]
card = pd.concat([index, card], 1)
card2 = pd.concat([index, card2], 1)
card2.columns = card.columns

In [42]:
#篩選有效資料
card = card[card['卡片'].notnull()]
card2 = card2[card2['卡片'].notnull()]

#合併兩組card資料表
card = pd.concat([card, card2], 0)
card.reset_index(inplace=True, drop=True)

'''
【特別注意】
card2['卡片數量']都是整數，dtype為int64，與card合併後Object內容物type不變造成split運算錯誤。(.str只能針對str物件)
合併到card的target項目同理。
這部分必須非常細心。

↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓處理↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
'''
card['卡片數量'] = card['卡片數量'].astype('str')

In [43]:
pd.DataFrame(card.isnull().sum(), columns=['Miss_Data']).T

,報價日期,報價廠商,專案名,數量,卡片,卡片材質,卡片印刷,卡片加工,卡片收縮,卡片數量,卡片單價,卡片新刀
Miss_Data,0,0,0,0,0,0,0,1,0,0,1,138


In [44]:
#拋棄非目標的未加工道林紙
card = card.drop((card[card['卡片加工'].isnull()].index[0]))

In [45]:
pd.DataFrame(card.isnull().sum(), columns=['Miss_Data']).T

,報價日期,報價廠商,專案名,數量,卡片,卡片材質,卡片印刷,卡片加工,卡片收縮,卡片數量,卡片單價,卡片新刀
Miss_Data,0,0,0,0,0,0,0,0,0,0,1,137


In [46]:
# card['卡片新刀'].unique()
#整理新刀欄位

label = {np.nan:0, '有':1, '特製刀膜':1, '擋板':1}
card['卡片新刀'] = card['卡片新刀'].map(label)

In [47]:
#拆解卡片尺寸

card_in = card['卡片'].str.split('\n', expand=True)
card_in.columns=['第1尺寸', '第2尺寸', '第3尺寸']
card_in = card_in.fillna('0x0')

for i, x in enumerate(card_in.columns):
    card_in['第'+str(i+1)+'尺寸面積'] = card_in[x].str.split('x', expand=True)[0].astype('float64')\
                                    * card_in[x].str.split('x', expand=True)[1].astype('float64')
card_in.head()

,第1尺寸,第2尺寸,第3尺寸,第1尺寸面積,第2尺寸面積,第3尺寸面積
0,63x88,0x0,0x0,5544.0,0.0,0.0
1,63x88,0x0,0x0,5544.0,0.0,0.0
2,63x88,0x0,0x0,5544.0,0.0,0.0
3,63x88,0x0,0x0,5544.0,0.0,0.0
4,63x88,0x0,0x0,5544.0,0.0,0.0


In [48]:
#拆解卡片數量

card_inn = card['卡片數量'].str.split('\n', expand=True)
card_inn.columns=['第1數量', '第2數量', '第3數量']
card_inn = card_inn.fillna(0).astype('float64')
card_inn.head()

,第1數量,第2數量,第3數量
0,40.0,0.0,0.0
1,80.0,0.0,0.0
2,80.0,0.0,0.0
3,80.0,0.0,0.0
4,80.0,0.0,0.0


In [49]:
card_inn['總使用面積'] = card_in['第1尺寸面積'] * card_inn['第1數量'] + \
                    card_in['第2尺寸面積'] * card_inn['第2數量'] + \
                    card_in['第3尺寸面積'] * card_inn['第3數量']
card_inn.head()

,第1數量,第2數量,第3數量,總使用面積
0,40.0,0.0,0.0,221760.0
1,80.0,0.0,0.0,443520.0
2,80.0,0.0,0.0,443520.0
3,80.0,0.0,0.0,443520.0
4,80.0,0.0,0.0,443520.0


In [50]:
#合併拆解後資料

card = pd.concat([card, card_in], 1)
card = pd.concat([card, card_inn], 1)
del card_in, card_inn
card = card.drop(columns=['報價廠商', '卡片', '卡片數量'])
card.head()

,報價日期,專案名,數量,卡片材質,卡片印刷,卡片加工,卡片收縮,卡片單價,卡片新刀,第1尺寸,第2尺寸,第3尺寸,第1尺寸面積,第2尺寸面積,第3尺寸面積,第1數量,第2數量,第3數量,總使用面積
0,2019-12-17,MOTT,500.0,350g中華寧波,4c/4c,撲克油+壓紋,收縮,58.0,0,63x88,0x0,0x0,5544.0,0.0,0.0,40.0,0.0,0.0,221760.0
1,2019-12-05,原子怪獸,1000.0,310g日本銅西,4c/4c,撲克油+壓紋,2收縮,39.0,0,63x88,0x0,0x0,5544.0,0.0,0.0,80.0,0.0,0.0,443520.0
2,2019-12-05,原子怪獸,1500.0,310g日本銅西,4c/4c,撲克油+壓紋,2收縮,37.0,0,63x88,0x0,0x0,5544.0,0.0,0.0,80.0,0.0,0.0,443520.0
3,2019-12-05,原子怪獸,2000.0,310g日本銅西,4c/4c,撲克油+壓紋,2收縮,35.2,0,63x88,0x0,0x0,5544.0,0.0,0.0,80.0,0.0,0.0,443520.0
4,2019-12-05,原子怪獸,4000.0,310g日本銅西,4c/4c,撲克油+壓紋,2收縮,29.2,0,63x88,0x0,0x0,5544.0,0.0,0.0,80.0,0.0,0.0,443520.0


In [51]:
#拆解日期
import datetime
card['報價日期'] = card['報價日期'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
card['報價年份'] = card['報價日期'].apply(lambda x: datetime.datetime.strftime(x, '%Y')).astype('int64')
card['報價月份'] = card['報價日期'].apply(lambda x: datetime.datetime.strftime(x, '%m')).astype('int64')
card['報價日'] = card['報價日期'].apply(lambda x: datetime.datetime.strftime(x, '%d')).astype('int64')

card = card.drop(columns=['報價日期'])

In [52]:
#Label encoding

label = ['專案名', '卡片材質', '卡片印刷', '卡片加工', '卡片收縮', '第1尺寸', '第2尺寸', '第3尺寸']
for x in label:
    card[x] = card[x].astype('category').cat.codes

In [53]:
card.columns

Index(['專案名', '數量', '卡片材質', '卡片印刷', '卡片加工', '卡片收縮', '卡片單價', '卡片新刀', '第1尺寸',
       '第2尺寸', '第3尺寸', '第1尺寸面積', '第2尺寸面積', '第3尺寸面積', '第1數量', '第2數量', '第3數量',
       '總使用面積', '報價年份', '報價月份', '報價日'],
      dtype='object')

In [54]:
'''
本次執行還在同年份不使用年份欄位
'''
card = card[['專案名', '數量', '卡片材質', '卡片印刷', '卡片加工', '卡片收縮', '卡片新刀', '第1尺寸',
       '第2尺寸', '第3尺寸', '第1尺寸面積', '第2尺寸面積', '第3尺寸面積', '第1數量', '第2數量', '第3數量',
       '總使用面積', '報價年份', '報價月份', '報價日', '卡片單價']]
card.head()

,專案名,數量,卡片材質,卡片印刷,卡片加工,卡片收縮,卡片新刀,第1尺寸,第2尺寸,第3尺寸,...,第2尺寸面積,第3尺寸面積,第1數量,第2數量,第3數量,總使用面積,報價年份,報價月份,報價日,卡片單價
0,5,500.0,8,0,5,6,0,13,0,0,...,0.0,0.0,40.0,0.0,0.0,221760.0,2019,12,17,58.0
1,19,1000.0,4,0,5,1,0,13,0,0,...,0.0,0.0,80.0,0.0,0.0,443520.0,2019,12,5,39.0
2,19,1500.0,4,0,5,1,0,13,0,0,...,0.0,0.0,80.0,0.0,0.0,443520.0,2019,12,5,37.0
3,19,2000.0,4,0,5,1,0,13,0,0,...,0.0,0.0,80.0,0.0,0.0,443520.0,2019,12,5,35.2
4,19,4000.0,4,0,5,1,0,13,0,0,...,0.0,0.0,80.0,0.0,0.0,443520.0,2019,12,5,29.2


# 預測

In [55]:
'''
改動 預測目標
'''
target = card[card['卡片單價'].isnull()]
card = card.dropna()

In [56]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV

### RandomForestRegression

In [57]:
rfModel = RandomForestRegressor()
rfModel.fit(card.iloc[:, : -1], card.iloc[:, -1])
cross_val_score(rfModel, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean()

0.5707084703178478

In [58]:
submit_rf = rfModel.predict(target.iloc[:, :-1])
submit_rf

array([119.133])

### GradientBoostingRegression

In [59]:
gbModel = GradientBoostingRegressor()
gbModel.fit(card.iloc[:, :-1], card.iloc[:, -1])
cross_val_score(gbModel, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean()

0.7274291442765809

In [60]:
submit_gb = gbModel.predict(target.iloc[:, :-1])
submit_gb

array([113.86492159])

### XGB

In [61]:
xgb = XGBRegressor()
xgb.fit(card.iloc[:, :-1], card.iloc[:, -1])
cross_val_score(xgb, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean()

[01:46:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:46:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:46:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:46:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:46:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:46:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.6982010053264572

In [62]:
submit_xgb = xgb.predict(target.iloc[:, :-1])
submit_xgb

array([107.64093], dtype=float32)

### LightGBM

In [63]:
lgbm = LGBMRegressor(objective='regression')
lgbm.fit(card.iloc[:, :-1], card.iloc[:, -1])
cross_val_score(lgbm, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean()

-1.8380804262150185

In [64]:
submit_lgbm = lgbm.predict(target.iloc[:, :-1])
submit_lgbm

array([97.62648978])

# Submit

In [73]:
submit = []
submit.append(submit_rf)
submit.append(submit_gb)
submit.append(submit_xgb)
submit.append(submit_lgbm)
pd.DataFrame(submit, index=pd.Index(['rfModel', 'gbModel', 'xgbModel', 'lgbmModel']), columns=['卡片1預測'])

,卡片1預測
rfModel,119.133000
gbModel,113.864922
xgbModel,107.640930
lgbmModel,97.626490


----
# Hyperparameter

In [ ]:
param_grid = {'learning_rate':[0.1, 0.3, 0.5],
              'n_estimators':[100, 200, 300],
              'subsample':[1.0, 0.5, 0.3, 0.1],
              'min_samples_split':[2, 5, 10],
              'min_samples_leaf':[1, 2, 4],
              'max_depth':[3, 6, 8],
              'tol':[0.01, 0.05, 0.1]}
gridsearch = GridSearchCV(gbModel, param_grid, n_jobs=-1, verbose=1)
grid_result = gridsearch.fit(card.iloc[:, :-1], card.iloc[:, -1])

print('Best Accuracy : {}\nUsing {}'.format(grid_result.best_score_, grid_result.best_params_))

In [67]:
hyper_gbModel = GradientBoostingRegressor(learning_rate=0.3,
                                          max_depth=6,
                                          min_samples_leaf=1,
                                          min_samples_split=5,
                                          n_estimators=100,
                                          subsample=1.0, tol=0.05)
hyper_gbModel.fit(card.iloc[:, :-1], card.iloc[:, -1])
cross_val_score(hyper_gbModel, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean()

0.6870953493893845

In [68]:
submit_hyper_gbModel = hyper_gbModel.predict(target.iloc[:, :-1])
submit_hyper_gbModel

array([87.28867591])

In [ ]:
param_grid = {'n_estimators':[100, 200, 300],
              'min_child_weight':[1, 5, 10],
             'gamma':[0.5, 1, 1.5, 2, 5],
             'subsample':[0.6, 0.8, 1.0],
             'max_depth':[3, 4, 5]}
gridsearch = GridSearchCV(xgb, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_result = gridsearch.fit(card.iloc[:, :-1], card.iloc[:, -1])
print('Best Accuracy : {}\nUsing {}'.format(grid_result.best_score_, grid_result.best_params_) )

In [69]:
hyper_xgb = XGBRegressor(gamma=1.5,
                         max_depth=3,
                         min_child_weight=1,
                         n_estimators=300,
                         subsample=0.6)
hyper_xgb.fit(card.iloc[:, :-1], card.iloc[:, -1])
cross_val_score(hyper_xgb, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean()

[01:46:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:46:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:46:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:46:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:46:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:46:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.7465694787582813

In [70]:
submit_hyper_xgb = hyper_xgb.predict(target.iloc[:, :-1])
submit_hyper_xgb

array([113.597824], dtype=float32)

In [74]:
submit.append(submit_hyper_gbModel)
submit.append(submit_hyper_xgb)
pd.DataFrame(submit, index=pd.Index(['rfModel', 'gbModel', 'xgbModel', 'lgbmModel', 'hyper_gbModel', 'hyper_xgbModel']),
             columns=['卡片1預測'])

,卡片1預測
rfModel,119.133000
gbModel,113.864922
xgbModel,107.640930
lgbmModel,97.626490
hyper_gbModel,87.288676
hyper_xgbModel,113.597824


In [76]:
#追加信心水準
level = [cross_val_score(rfModel, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean(),
         cross_val_score(gbModel, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean(),
         cross_val_score(xgb, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean(),
         cross_val_score(lgbm, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean(),
         cross_val_score(hyper_gbModel, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean(),
         cross_val_score(hyper_xgb, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean()]

df_level = pd.DataFrame(level, columns=['預測信心'])
df = pd.DataFrame(submit, columns=['卡片1預測'])
df = pd.concat([df, df_level], 1)
df.index=pd.Index(['rfModel', 'gbModel', 'xgbModel', 'lgbmModel', 'hyper_gbModel', 'hyper_xgbModel'])
df

[01:47:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:47:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:47:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:47:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:47:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:47:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:47:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:47:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:47:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

,卡片1預測,預測信心
rfModel,119.133000,0.622540
gbModel,113.864922,0.707779
xgbModel,107.640930,0.698201
lgbmModel,97.626490,-1.838080
hyper_gbModel,87.288676,0.686856
hyper_xgbModel,113.597824,0.746569
